In [ ]:
"""
Created on Sun Aug 23 16:55:01 2020

@author: Giannis

sources: 
datacamp.com/community/tutorials/principal-component-analysis-in-python
rpubs.com/Sharon_1684/454441
kaggle.com/olaniyan/image_classification_using_knn

Θα πρέπει να γινει upload το συμπιεσμένο αρχείο των εικόνων
"""
from google.colab import files
#uploaded = files.upload()
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['images.zip']), 'r')
data.extractall()
data.printdir()
# To decode the files
import pickle
# For array manipulations
import time
import numpy as np
# To make one-hot vectors
from keras.utils import np_utils
# To plot graphs and display images
from matplotlib import pyplot as plt
path = "images/" 

from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn.svm import SVC

size = 100,100;
import cv2
import seaborn as sns
import re
import os
import imageio
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import pandas as pd
y_tr = 0
#fig, ax = plt.subplots()

# 5-fold #
#kf = StratifiedKFold(n_splits=5, random_state=123)

def convert_word_to_label(word):

    if word == 'W':
        return 0
    elif word == 'F':
        return 1
    elif word == 'S':
        return 2
    

def PCA_ImageSpaceVisualization(x_train):
    np.min(x_train),np.max(x_train)
    
    x_tr = np.asarray(x_train)/255.0
    x_tr.shape
      
    x_train_flat = x_tr.reshape(-1,30000)
    print (x_train_flat.shape)
    feat_cols = ['pixel'+str(i) for i in range(x_train_flat.shape[1])]
    df_cifar = pd.DataFrame(x_train_flat,columns=feat_cols)
    df_cifar['label'] = y_tr
    #print('Size of the dataframe: {}'.format(df_cifar.shape))
    #print(df_cifar.head())
    pca_cifar = PCA(n_components=2)
    principalComponents_cifar = pca_cifar.fit_transform(df_cifar.iloc[:,:-1])
    
    
    principal_cifar_Df = pd.DataFrame(data = principalComponents_cifar, columns = ['principal component 1', 'principal component 2'])
    principal_cifar_Df['y'] = y_tr
    #print(principal_cifar_Df.head())
    
    plt.figure(figsize=(16,10))
    sns.scatterplot(
    x="principal component 1", y="principal component 2",
    hue="y",
    palette=sns.color_palette("hls", 3),
    data=principal_cifar_Df,
    legend="full",
    alpha=0.6,s=150
)
    
    plt.figure(figsize=(10, 10))
    

    #linear SVM

    ss = StandardScaler()
    # run this on our feature matrix
    x_train_flat2 = ss.fit_transform(x_train_flat)
    
    pca = PCA(.95)
    # use fit_transform to run PCA on our standardized matrix
    pca.fit_transform(x_train_flat2)
    #print(pca.n_components_ )   
  
    X = pd.DataFrame(x_train_flat2)
    y = pd.Series(y_tr)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)
    svm = SVC(kernel='linear', probability=True, random_state=42)
    knn = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=1))
    # fit model
    svm.fit(X_train,y_train)
    knn.fit(X_train,y_train)
   # generate predictions
    
    #y_pred = svm.predict(X_test)
    y_pred = knn.predict(X_test)
    # calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print('Model accuracy for 1-nn is: ', accuracy)
    res_svm = cross_val_score(svm, X, y, cv=5)
    res_knn = cross_val_score(knn, X, y, cv=5)
    
    avg_score_svm = np.mean(res_svm)
    avg_score_knn = np.mean(res_knn)
    print("SVM k-fold",res_svm,"avg:",avg_score_svm)
    print("1NN k-fold",res_knn,"avg:",avg_score_knn)
    #scoring = ['accuracy'] 
          
    return principal_cifar_Df

def loadImages(path):
    X = []
    y = []

    for r, d, f in os.walk(path):
        for image in f:
            if '.jpg' in image:
                image_path = os.path.join(r, image)
                img = cv2.imread(image_path)
                new_img = cv2.resize(img, (size), interpolation=cv2.INTER_CUBIC)
                X.append(new_img)
                word = re.split('(\d+)',image)[0]
                y.append(convert_word_to_label(word))
    return X, y
if __name__ == '__main__':

    X, y = loadImages(path)
    y_tr = y
    a = PCA_ImageSpaceVisualization(X)
    #print (a)
    


**Απαντήσεις  στα ερωτήματα του 2**

Χρησιμοποιώντας την PCA στο παράδειγμά μας καταφέραμε να εξάγουμε μια αναπαράσταση χαμηλών διαστάσεων η οποία μπορεί να χρησιμοποιηθεί στη θέση των αρχικών δεδομένων για ταξινόμηση και οπτικοποίηση (προβολή δεδομένων στο χώρο μικρότερης διάστασης)
    
Οι εικόνες που οπτικοποιούνται σε κοντινή απόσταση σε αυτό το χώρο δύο διαστάσεων έχουν κάποια κοινά χαρακτηριστικά, για παράδειγμα διαθέτουν παρόμοια χρώματα και μπορεί να έχουν ληφθεί την ίδια εποχή του χρόνου.
  
Γενικά τα δεδομένα στο χώρο των δύο διαστάσεων μπορεί να μην είναι γραμμικώς διαχωρίσιμα,όπως φαίνεται στο παράδειγμα για τις εικόνες που ανήκουν στις δύο απο τις τρείς εποχές που μελετάμε (άνοιξη, φθινόπωρο). Για να γίνει ακριβής διαχωρισμός ενδεχομένως χρειάζεται η προβολή των δεδομένων σε μεγαλύτερο χώρο διαστάσεων.
  
Όταν απέχουν πολύ οι εικόνες στην οπτικοποίηση αυτό σημαίνει οτι απεικονίζουν κάτι πολύ διαφορετικό και δεν θα μπορούσαν να ομαδοποιηθούν στην ίδια κατηγορία απο τον ταξινομητή. Δείχνει η απόσταση και την ύπαρξη μεγάλης διακύμανσης στα δεδομένα. Η ερμηνευόμενη διακύμαση διατηρείται στα δεδομένα με τις συνιστώσες που έχουμε διατηρήσει.

**Απαντήσεις στα ερωτήματα του 3**
   
Mαθηματικός ορισμός της ακρίβειας (accuracy)
Accuracy = Number of correct predictions/Total number of predictions = 
TP+TN/ TP + TN + FP + FN, όπου TP = Positive True, FP = False Positive FN = False Negative
                            TN = True Negative
   
O ταξινομητής που φαίνεται να έχει την καλύτερη επίδοση με βάση τα αποτελέσματα στα πειράματα είναι ο ταξινομηνής linear SVM. Ο λόγος είναι ότι η τιμή της υπερπαραμέτρου k στον αλγόριθμο Κ-nn δέν είναι η βέλτιστη. Δηλαδή λαμβάνουμε υπόψη έναν μόνο γείτονα για την ταξινόμηση κι αυτό δεν είναι αρκετά αποδοτικό.
    
Mε την εντολή, για παράδειγμα pca = PCA(.95) μπορούμε να προσδιορίσουμε  
τη διάσταση των χαρακτηριστικών που θέλουμε να εξάγουμε μέσω της PCA. Εκτυπώνοντας τον πίνακα που προκύπτει απο την επεξεργασια των δεδομένων μπορούμε να δούμε τις νεες διαστάσεις. Δηλαδή επιλέγουμε αριθμό κύριων διαστάσεων έτσι ώστε να κρατήσουμε το 95% της διακυμανσης στο παράδειγμα. Εναλλακτικά, pca = PCA(n_components=αριθμος διαστασεων)

In [ ]:
"""
ερωτημα 2
@author: Giannis
RegNMF (Regulated non matrix factorization)
Is an alternative solution for the generalized PCA and regularized PCA. 
This keeps  the non negative numbers of the input array and has easier interpretable components.
---Returns---
The non negative tables W (d x k) and C (k X N) fully representing the X.
C   :   matrix of components
W   :   is the base
---Inputs---
X   :   non negative table dxN = (500 x 1000). 
        the input array for decomposition
k   :   integer 
        the number of components
lambda_ :   float 
            the regularization parameter
epsilon :   float
            the lowest threshold for the termination

"""
import numpy as np

def gradientDescent_C(X, W, C):
    W_transp = np.transpose(W)
    nom = np.dot(W_transp,X)
    denom = np.linalg.multi_dot([W_transp, W, C])
    div = np.divide(nom, denom)
    C = np.multiply(C, div)
    return C


def gradientDescent_W(X, W, C):
    C_transp = np.transpose(C)
    nom = np.dot(X, C_transp)
    denom = np.linalg.multi_dot([W, C, C_transp])
    div = np.divide(nom, denom)
    W = np.multiply(W, div)
    return W


def error_(X, W, C):
    dist = np.linalg.norm(np.subtract(X, np.dot(W, C)))
    X_norm_2 = np.linalg.norm(X)
    error = np.divide(dist, X_norm_2)
    return error


def RegNMF(X, k, lambda_, epsilon):
    D = 500
    N = 1000
    iters = 500
    # n = 0.001 # step
    W = abs(np.random.rand(D, k))     # random initializions on W, C
    C = abs(np.random.rand(k, N))
    for e in epsilon:
        for t in range(1, iters):
            W_prev = W
            C_prev = C
            C = gradientDescent_C(X, W, C)
            W = gradientDescent_W(X, W, C)
            error_t_prev = error_(X, W_prev, C_prev)
            error_t = error_(X, W, C)
            # print("error_t : ", error_t)
            # print("error_t_prev : ", error_t_prev)

            if abs(np.subtract(error_t, error_t_prev)) < e:
                yield W, C, t, e
                break

X = abs(np.random.rand(500, 1000))
k = 10
lambda_ = 0.1
epsilon = [0.01, 0.001, 0.0001]

gen = RegNMF(X, k, lambda_, epsilon)
for W, C, t, e in gen:
    print("W : ", W)
    print("C : ", C)
    print("Run for {} iterations and epsilon {} \n".format(t, e))
    
    


Συμφωνα με τον αλγόριθμο παρακολουθούμε το σφάλμα ανακατασκευής και αν η τιμή του ανάμεσα
σε δυο διαδοχικές επαναλήψεις είναι μικρότερη απο ενα κατώφλι ε τερματίζουμε τον αλγόριθμο
Το πείραμα δείχνει οτι σε περίπτωση που επιλέξουμε πολυ μικρό ε χρειάζονται πολύ περισσότερες
επαναλλήψεις (40+ στο παράδειγμα) προκειμένου να συγκλίνει και να τερματίσει ο αλγόριθμος, σε σχέση με μεγαλύτερες τιμές του ε